The idea is to focus on just the generative portion of the model and test new models as they are available. We will start by testing Marco o1

In [ ]:
import json
from typing import Dict, List, Optional, Tuple

def extract_examples(data: List[Dict], index: int) -> Tuple[str, List[Tuple[str, str]]]:
    """
    Extract the base finding and reference examples from the evaluation results.
    
    Args:
        data: List of dictionaries containing the evaluation results
        index: Index of the item to extract
        
    Returns:
        Tuple containing:
            - Base finding string
            - List of tuples, each containing (finding, conclusion) pairs from reference examples
    """
    item = data[index]
    base_finding = item['finding']
    
    reference_examples = [
        (example['finding'], example['conclusion'])
        for example in item['reference_examples']
    ]
    
    return base_finding, reference_examples

def create_prompt(
    base_finding: str,
    reference_examples: List[Tuple[str, str]],
    base_prompt: Optional[str] = None
) -> str:
    """
    Create a formatted prompt combining the base finding and reference examples.
    
    Args:
        base_finding: The primary finding to generate conclusions for
        reference_examples: List of (finding, conclusion) pairs to use as examples
        base_prompt: Optional custom instructions for the prompt
        
    Returns:
        Formatted prompt string
    """
    if base_prompt is None:
        base_prompt = """Please generate the "Conclusions" section of a medical report based on the provided "Findings" section. 
Below are several example pairs of "Findings" and their corresponding "Conclusions" to help guide the format and style of your response.

After reviewing these examples, please generate an appropriate conclusion for the new finding provided at the end."""

    # Format reference examples
    examples_text = "\n\n".join([
        f"Example {i+1}:\n"
        f"Findings:\n{finding}\n\n"
        f"Conclusion:\n{conclusion}"
        for i, (finding, conclusion) in enumerate(reference_examples)
    ])
    
    # Combine all parts
    full_prompt = f"""{base_prompt}

{examples_text}

New Finding to generate conclusion for:
{base_finding}"""
    
    return full_prompt

# Example usage:
if __name__ == "__main__":
    # Load data from file
    with open("evaluation_results.json", "r") as f:
        data = json.load(f)
    
    # Extract examples for index 0
    base_finding, reference_examples = extract_examples(data, 0)
    
    # Create prompt
    prompt = create_prompt(base_finding, reference_examples)
    print(prompt)